In [36]:
import pandas as pd
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, mean_squared_error
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
import zipfile
import os

In [37]:
# Load the dataset
df = pd.read_csv("pima_cleaned.csv")

In [38]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    float64
 1   Glucose                   768 non-null    float64
 2   BloodPressure             768 non-null    float64
 3   SkinThickness             768 non-null    float64
 4   Insulin                   768 non-null    float64
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    float64
 8   Outcome                   768 non-null    int64  
dtypes: float64(8), int64(1)
memory usage: 54.1 KB
None


In [32]:
# Number of columns excluding the last one
num_cols = len(df.columns) - 1

# Check each column
for i in range(num_cols):
    col = df.iloc[:, i]  # Select column i
    min_val = col.min()
    max_val = col.max()

    # Check if normalized
    if min_val < 0 or max_val > 1:
        print(f"Column {df.columns[i]} not normalized. Min: {min_val}, Max: {max_val}")
    else:
        print(f"Column {df.columns[i]} normalized. Min: {min_val}, Max: {max_val}")

Column Pregnancies normalized. Min: 0.0, Max: 1.0
Column Glucose normalized. Min: 0.0, Max: 1.0
Column BloodPressure normalized. Min: 0.0, Max: 0.9999999999999998
Column SkinThickness normalized. Min: 0.0, Max: 1.0
Column Insulin normalized. Min: 0.0, Max: 1.0
Column BMI normalized. Min: 0.0, Max: 0.8425357873210637
Column DiabetesPedigreeFunction normalized. Min: 0.0, Max: 0.9999999999999998
Column Age normalized. Min: 0.0, Max: 1.0


In [40]:
# تقسیم داده‌ها به ویژگی‌ها و بردار هدف (Split data into features and target)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [41]:
# محاسبه فاصله اقلیدسی (Calculate Euclidean distance)
distance_matrix = euclidean_distances(X)